In [65]:
'''
Notebook para asociar una velocidad de viento a una curva de potencia ideal con un rango de error por definir
'''

'\nNotebook para asociar una velocidad de viento a una curva de potencia ideal con un rango de error por definir\n'

# Imports

In [2]:
%matplotlib notebook
import pandas as pd
import numpy as np
#importa las fuciones que cree desde otro notebook (instalar modulo ipynb)
import ipynb.fs.defs.my_funcs_clusters as myfunc
import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter
from matplotlib import rc, font_manager
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython import display as dpy
import ipynb.fs.defs.my_funcs_clusters as myfunc
import matplotlib

In [3]:
pd.set_option('display.max_rows', 100)
#matplotlib.use('nbAGG')

In [4]:
import importlib
importlib.reload(myfunc)

<module 'ipynb.fs.defs.my_funcs_clusters' (C:\Users\mungu\Documents\GitHub\aero\my_funcs_clusters.ipynb)>

# Procesamiento de datos

In [5]:
data_path ='datos/DP-PB01-2005.xlsx'
mf_pow_curve_path ='datos/DeWind d8.2.csv'
df_wind_dir, df_comp_vel,df_mf_curve = myfunc.proc_dat_bcs(data_path,mf_pow_curve_path)


Inciando procesamiento de datos...

Total de registros: 52560
Numero de filas sin datos
day      0.0
hour     0.0
wdir     0.0
vwind    0.0
dtype: float64
Fin del procesamiento de datos


## Interpolación de la curva del fabricante

In [6]:
# puntos de la curva del fabricante
plt.figure(1)
plt.scatter(df_mf_curve.index,df_mf_curve.pw,s=1)
plt.show()

<IPython.core.display.Javascript object>

In [7]:
v_min =0
#maximo de la curva de potencia del fabricante
v_max = df_mf_curve.index.max()
df_mfcurve_interp = myfunc.interp_pow_curve(df_mf_curve,wind_min_max=(v_min,v_max))
plt.figure(5,figsize=(10,8))
plt.scatter(df_mfcurve_interp.index,df_mfcurve_interp.pw,s=2)
plt.show()

<IPython.core.display.Javascript object>

## Crear un dataframe viento-potencia del fabricante

In [8]:
df_mfcurve_interp[df_mfcurve_interp.index==0.1].pw.values

array([0.])

In [9]:
#voy a usar vwind redondeado a un decimal
#el porcentaje de error lo defino despues
#columna de potencias segun la velocidad del viento
#redondeando los valores de velocidad de viento a un decimal
df_wind_dir['vwind'] = np.round(df_wind_dir.vwind,1)
col_pw_mf = [df_mfcurve_interp[df_mfcurve_interp.index==v].pw.values[0] for
             v in  df_wind_dir.vwind]

#data frame que contiene los datos de viento reales y la potencia
#generada por esos datos de viento si evaluara la curva de
#potencia con ellos
df_wind_pow_ideal = pd.DataFrame(df_wind_dir.vwind)
df_wind_pow_ideal['pw']=col_pw_mf

# Encontrar la relación viento-potencia
Quiero saber por cada x cantidad de viento, cuanta potencia sube

In [10]:
fechaini = '2005-01-01 00:00:00'
fechafin= '2005-01-01 23:59:59'
df_wind_pow_ideal_1Dia = df_wind_pow_ideal.loc[fechaini:fechafin]
df_wind_pow_ideal1Dia_ord=df_wind_pow_ideal_1Dia.sort_values(by=['vwind'])
plt.plot(df_wind_pow_ideal1Dia_ord.vwind,df_wind_pow_ideal1Dia_ord.pw)

In [11]:
#porque son las tres de la mañana, se salta el primer dia
#HORRIBLE CODIGO
rango_fechas = pd.date_range(df_wind_pow_ideal.index[0],df_wind_pow_ideal.index[-1])

###########   widgets #########

sslider =widgets.SelectionSlider(
    options=rango_fechas,
    description='Dia:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
)

output3= widgets.Output()

display(sslider, output3)

############ funcion y ploteo###############
plt.ioff()
fig = plt.figure(figsize=(10,7))
axes = fig.add_subplot(111)

hfig = display(fig, display_id=True)
def on_value_change3(change,):
    with output3:
        #no sirve si la fecha y hora exacta no existe
        dia=df_wind_pow_ideal.loc[change['new']:change['new']+pd.Timedelta(1,'day')]
        dia_ord = dia.sort_values(by='vwind')
        #ax.set_xdata(dia_ord.vwind)
        #ax.set_ydata(dia_ord.pw)
        axes.cla()
        axes.plot(dia_ord.vwind,dia_ord.pw)
        axes.set_xlim([0,30])
        axes.set_ylim([-40e3,2.2e6])
        fig.canvas.draw()
        hfig.update(fig)

sslider.observe(on_value_change3, names='value')

SelectionSlider(continuous_update=False, description='Dia:', options=(Timestamp('2005-01-01 00:10:00', freq='D…

Output()

<IPython.core.display.Javascript object>

In [12]:
#Recolectando datos
#numero de dias
d1 =df_wind_pow_ideal.index[0]
d2=df_wind_pow_ideal.index[-1]
n =d2-d1
ndias_total=n.days


# Observando la variacion entre velocidades de viento iguales y potencia

In [13]:
#buscar valores unicos de velocidad de viento
vv_unic= np.sort(df_wind_pow_ideal.vwind.unique())
a =vv_unic
vvmin = df_wind_dir.vwind.min()
vvmax = df_wind_dir.vwind.max()
b =np.round(np.arange(vvmin,vvmax,0.1),decimals=1)
c= list(set(a)  ^ set(b))
print('Deberian ser %d velocidades, pero son %d'%(len(b),len(a)))
print('Faltan %d velocidades'%(len(c)))
print('Las velocidades que faltan son:')
print(*c, sep = ", ") 


Deberian ser 180 velocidades, pero son 169
Faltan 13 velocidades
Las velocidades que faltan son:
18.0, 16.4, 16.6, 16.9, 17.1, 17.4, 17.5, 17.9, 16.3, 16.8, 17.3, 17.7, 17.8


In [14]:
#d = df_wind_pow_ideal.loc[df_wind_pow_ideal.vwind==vv_unic[100]]


In [15]:
#porque son las 5:30 de la mañana, se salta el primer dia
#HORRIBLE CODIGO
#no se ordenand las velocidades de viento por potencia,debe ser por hora de lectura

###########   widgets #########

sslidervv =widgets.SelectionSlider(
    options=vv_unic,
    description='Velocidad:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
)

outputvv= widgets.Output()

display(sslidervv, outputvv)

############ funcion y ploteo###############
plt.ioff()
fig = plt.figure(figsize=(10,7))
axes = fig.add_subplot(111)

hfig = display(fig, display_id=True)
def on_value_changevv(change,):
    with outputvv:
        #no sirve si la fecha y hora exacta no existe
        vv_pw=df_wind_pow_ideal.loc[df_wind_pow_ideal.vwind==change['new']]
        df_describe =vv_pw.describe()
        #convertir el describe a texto y ponerlo en formato cientifico para la tabla
        describe_sci =[[format(element,'.2E') for element in el] for el in df_describe.values.tolist()]

        axes.cla()
        axes.plot(vv_pw.index,vv_pw.pw)
        axes.table(describe_sci, colLabels=df_describe.columns,rowLabels=df_describe.T.columns)
        axes.tick_params(axis='x', pad=-20)
        axes.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

        #axes.set_xlim([0,30])
#         axes.set_ylim([-40e3,2.2e6])
        fig.canvas.draw()
        hfig.update(fig)
sslidervv.observe(on_value_changevv, names='value')

SelectionSlider(continuous_update=False, description='Velocidad:', options=(0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6,…

Output()

<IPython.core.display.Javascript object>

In [16]:
h =df_wind_pow_ideal.loc[df_wind_pow_ideal.vwind==7]
a =h.describe()
pd.set_option('display.float_format', lambda x: '%.3f' % x)
a
b=a.values

In [17]:
b.tolist()

[[466.0, 466.0],
 [7.0, 781000.0],
 [0.0, 0.0],
 [7.0, 781000.0],
 [7.0, 781000.0],
 [7.0, 781000.0],
 [7.0, 781000.0],
 [7.0, 781000.0]]

In [18]:
d =list(map(len,b.tolist()))
d


[2, 2, 2, 2, 2, 2, 2, 2]

In [19]:
[[format(element,'1.2E') for element in el] for el in b.tolist()]


[['4.66E+02', '4.66E+02'],
 ['7.00E+00', '7.81E+05'],
 ['0.00E+00', '0.00E+00'],
 ['7.00E+00', '7.81E+05'],
 ['7.00E+00', '7.81E+05'],
 ['7.00E+00', '7.81E+05'],
 ['7.00E+00', '7.81E+05'],
 ['7.00E+00', '7.81E+05']]

# Ubicar transiciones en curva de potencia

In [20]:
import importlib
importlib.reload(myfunc)

<module 'ipynb.fs.defs.my_funcs_clusters' (C:\Users\mungu\Documents\GitHub\aero\my_funcs_clusters.ipynb)>

## clustering

In [21]:
#####################

n_clusters = 15
n_subclu=0
clusters_wind_pow = myfunc.KMData()
clusters_wind_pow.dataframe_to_cluster(df_comp_vel, n_clusters=n_clusters,n_sub_clusters=n_subclu,
    clusters_data='wind', subclusters_data='pw',dfVP=df_wind_pow_ideal,pw_col_name='pw',
    wind_col_name='vwind')

#def dataframe_to_cluster(dfvxvy, n_clusters,dfVP =None, n_subclu=0, clusters_data=None,subclusters_data=None,
#                         datadir=None):
#crear un dataframe viento, direccion, cluster
#df_wind_pow_cl = df_wind_pow_ideal.copy()
#tomando las labels de kmeans y sumandole uno por que empieza en 0
#ademas agregando la letra C y ordenando de menor v a mayor vel
#col_clnames = ['C' + str(clusters_wind.cl_ord[c]+1) for c in clusters_wind.kmeans_labels]
#df_wind_pow_cl['cluster'] = col_clnames
#del col_clnames



## Plot

### Plot solo viento

In [22]:
#solo viento
clsclord=('wind',None)
ploti = myfunc.PlotSubClusterInt()
ploti.create_plot(clusters_wind_pow.comp_vel, figsize=(20, 10), idx_centroids=clusters_wind_pow.idx_centroids,fign=208,
                  savepath='fig_art/',
                  showlBetz=False, showCent= True, showOpt= 'Numero')
plt.show()

Manufacturer power curve missing


AttributeError: 'NoneType' object has no attribute 'get_xticklabels'

In [ ]:
### Plot viento potencia ideal

In [23]:

#viento potencia
ploti = myfunc.PlotSubClusterInt()
ploti.create_plot(clusters_wind_pow.comp_vel,
                  dfclvp=clusters_wind_pow.dfclvp,
                  datavp =df_wind_pow_ideal,
                  dfMfgCurve = df_mf_curve,
                  cl_scl_order=('wind', 'pow'),
                  figsize=(20, 10),
                  idx_centroids=clusters_wind_pow.idx_centroids,
                  fign=208,
                  savepath='fig_art/',
                  showOpt= 'Numero')
plt.show()

<IPython.core.display.Javascript object>

In [24]:
'''
TO DO:
-en el idx_centroids los numeros de los clusters no llevan la C, en el idx_centroids_sc si
    la llevan, o cambiar todo a que la lleven o que no la lleven
-Cambiar assign, o ver como puedo poner un variable como nombre, que en lugar de pw
    sea pw_col_name
'''

from sklearn import cluster, datasets
from scipy.spatial.distance import cdist

class KMData:
    """
    Clase para tener un objeto con todos los datos de los clusters.
    """
    def __init__(self):
        self.kmeans_labels = None
        self.dfclvv = None
        self.cl_ord = None
        self.scl_ord= None
        self.scl_centroids = None
        self.idx_centroids = None
        self.idx_centroids_sc = None
        self.cl_centers = None
        self.dfclpw = None
        self.dfclvp= None

    #crear clusters a partir de dataframes
    def dataframe_to_cluster(self,dfvxvy, n_clusters,dfVP =None, n_sub_clusters=0, clusters_data=None,
                             subclusters_data=None, datadir=None,pw_col_name='pw', wind_col_name='vwind'):
        """
            Descripcion:
            Esta funcion toma diferentes dataframes de entrada agrupa los datos por cluster.

            :param wind_col_name: nombre de la columna "viento" del dataframe
            :param pw_col_name: nombre de la columna "potencia" del dataframe
            :param datadir: Datos de direccion
            :param subclusters_data: data to do subclustering [wind,pow,dir]
            :param clusters_data: data to do clustering [wind,pow,dir]
            :param dfvxvy: dataframe con columnas vx y vy
            :param dfVP: dataframe con columnas de magnitud de viento y potencia
            :param n_clusters: numero de clusters
            :param n_sub_clusters: Número de suclusters a calcular a partir de los n clusters calculados en un principio.
                      Si n_sub_clusters=0 no se calcula ningun subcluster. Por defecto n_sub=0


            :return cl_ord: un array que contiene el número de cluster n ordenado de menor a mayor
                de las magnitudes de la velocidad de viento sin tomar en cuenta la direccion
            :return dfclvv: dataframe donde las componentes de velocidad de viento vx y vy estan
                agrupadas por cluster
            :return dfclpw: dataframe donde la potencia esta agrupada por cluster
            :return dfclvp: dataframe donde la potencia y la magnitud de viento esta agrupada por
                cluster
        """

        #inicializacion de variables internas
        dfclpw = None
        dfclvp = None
        scl_ord = None
        scl_centroids = None
        idx_centroids_sc = None

        #check if dataframe with power values exist
        if dfVP is None:
            dfvp_is_empty = True
        else:
            dfvp_is_empty = False

        #'''- KMEANS -----------------------------------------------------------------------------------------'''
        if clusters_data=='wind':
            kmeans = cluster.KMeans(n_clusters=n_clusters,random_state=0).fit(dfvxvy)
            #buscar centroides en dataframes
            #https://stackoverflow.com/questions/42583995/get-the-centroid-row-index-from-k-means-clustering-using-sklearn
            min_dist = np.min(cdist(dfvxvy.values, kmeans.cluster_centers_, 'euclidean'), axis=1)#distancia minima a cada centroide
            Y = pd.DataFrame(min_dist, index=dfvxvy.index, columns=['PCTimeStamp'])
            Z = pd.DataFrame(kmeans.labels_, index=dfvxvy.index, columns=['cluster_ID'])
            #crea el dataframe
            PAP = pd.concat([Y,Z], axis=1)

        if clusters_data=='pow' and not dfvp_is_empty:

            kmeans = cluster.KMeans(n_clusters=n_clusters,random_state=0).fit(dfVP)
            #buscar centroides en dataframes
            min_dist = np.min(cdist(dfVP.values, kmeans.cluster_centers_, 'euclidean'), axis=1)
            Y = pd.DataFrame(min_dist, index=dfVP.index, columns=['PCTimeStamp'])
            Z = pd.DataFrame(kmeans.labels_, index=dfVP.index, columns=['cluster_ID'])
            PAP = pd.concat([Y,Z], axis=1)

        elif clusters_data=='pow' and  dfvp_is_empty:

            print('Power data is emtpy...')
            return None

        if clusters_data=='dir' and not dfvp_is_empty:
            kmeans = cluster.KMeans(n_clusters=n_clusters,random_state=0).fit(datadir)
            #buscar centroides en dataframes
            min_dist = np.min(cdist(dfVP.values, kmeans.cluster_centers_, 'euclidean'), axis=1)
            Y = pd.DataFrame(min_dist, index=dfVP.index, columns=['PCTimeStamp'])
            Z = pd.DataFrame(kmeans.labels_, index=dfVP.index, columns=['cluster_ID'])
            PAP = pd.concat([Y,Z], axis=1)

        elif clusters_data =='dir' and dfvp_is_empty:

            print('Clustering with only direction and no power data not implemented yet.')
            return None

        #ordend e los cluster por magnitud de vv
        cl_magni = np.zeros(n_clusters)

        for i in range(n_clusters):
            vx = dfvxvy.vx.values[kmeans.labels_ == i]
            vy = dfvxvy.vy.values[kmeans.labels_ == i]
            cl_magni[i] = np.mean(np.sqrt(vx**2 + vy**2))  #magnitud de la vv
        cl_ord = np.argsort(cl_magni.argsort()) #ver https://github.com/numpy/numpy/issues/8757

        grouped = PAP.groupby(['cluster_ID'])#agrupa por numero de clusters las distancias minimas
        idx_centroids = grouped.idxmin()#encuentra el indice de la distanciam minima
        #agregando columnas de velocidadd de viento y potencia al dataframe de los centroides
        vxvy=dfvxvy.loc[idx_centroids.PCTimeStamp].values
        if dfvp_is_empty:
            idx_centroids=idx_centroids.assign(vx=vxvy[:,0],vy=vxvy[:,1])
        else:
            vvpot = dfVP.loc[idx_centroids.PCTimeStamp].values
            idx_centroids=idx_centroids.assign(vx=vxvy[:,0],vy=vxvy[:,1],vwind=vvpot[:,0],pw=vvpot[:,1])
            idx_centroids.sort_values(by='vwind',inplace=True)# ordenar por velocidad de viento


        idx_centroids.reset_index(inplace=True) #que ya no sea el cluster id el indice
        idx_centroids.index.set_names('cluster_ID_ord',inplace=True)  #ponerle el nombre al indice ordenado
        idx_centroids.index+=1 #paraque el indice empieze en uno y no exista cluster 0




        ## CREAR MULTIINDICE ---------------------------------------------------------------'''
        #CLUSTER VX,VY:
        #https://stackoverflow.com/questions/37835508/how-to-do-multi-column-from-tuples
        #nombre de las columnas del dataframe
        if dfvp_is_empty:
            colheadvv = []
            for i in range(n_clusters):
                colheadvv.append(('C' + str(i + 1), 'vx'))
                colheadvv.append(('C' + str(i + 1), 'vy'))
            dfclvv = pd.DataFrame()

            for i in range(n_clusters):
                dfclvv = pd.concat([
                    dfclvv, dfvxvy.vx[kmeans.labels_ ==np.where(cl_ord ==i)[0][0]],
                    dfvxvy.vy[kmeans.labels_ == np.where(cl_ord ==i)[0][0]]
                ],
                    axis=1,
                    ignore_index=True)
            #crear multiindice
            dfclvv.columns = pd.MultiIndex.from_tuples(colheadvv)

        else:

            colheadvv = []
            colheadpw = []
            for i in range(n_clusters):
                colheadvv.append(('C' + str(i + 1), 'vx'))
                colheadvv.append(('C' + str(i + 1), 'vy'))
                colheadpw.append('C' + str(i + 1))
            dfclvv = pd.DataFrame()

            for i in range(n_clusters):
                dfclvv = pd.concat([
                    dfclvv, dfvxvy.vx[kmeans.labels_ ==np.where(cl_ord ==i)[0][0]],
                    dfvxvy.vy[kmeans.labels_ == np.where(cl_ord ==i)[0][0]]
                ],
                    axis=1,
                    ignore_index=True)
            #crear multiindice
            dfclvv.columns = pd.MultiIndex.from_tuples(colheadvv)

            #CLUSTER POTENCIA:
            dfclpw = pd.DataFrame()
            for i in range(n_clusters):
                dfclpw = pd.concat([dfclpw, dfVP[pw_col_name][kmeans.labels_ == np.where(cl_ord ==i)[0][0]]],
                                   ignore_index=True,
                                   axis=1)
            dfclpw.columns = colheadpw

            #CLUSTER VIENTO POTENCIA:

            colheadvp = []
            for i in range(n_clusters):
                colheadvp.append(('C' + str(i + 1), 'vwind'))
                colheadvp.append(('C' + str(i + 1), 'pw'))
            dfclvp = pd.DataFrame()

            for i in range(n_clusters):
                dfclvp = pd.concat([
                    dfclvp, dfVP[wind_col_name][kmeans.labels_ == np.where(cl_ord ==i)[0][0]],
                    dfVP[pw_col_name][kmeans.labels_ ==np.where(cl_ord ==i)[0][0]]] ,
                    axis=1,
                    ignore_index=True)
            #crear multiindice
            dfclvp.columns = pd.MultiIndex.from_tuples(colheadvp)

        if clusters_data=='direccion':
            #CLUSTER direccion viento
            colheadvp = []
            for i in range(n_clusters):
                colheadvp.append(('C' + str(i + 1), 'Dir'))
                colheadvp.append(('C' + str(i + 1), pw_col_name))
            dfcldv = pd.DataFrame()
            for i in range(n_clusters):
                dfcldv = pd.concat([
                    dfcldv, datadir[wind_col_name][kmeans.labels_ ==( cl_ord ==i)],
                    datadir.Dir[kmeans.labels_ == (cl_ord ==i)]],
                    axis=1,ignore_index=True)
            #crear multiindice
            dfcldv.columns = pd.MultiIndex.from_tuples(colheadvp)
        '''- CALCULAR SUBCLUSTERS ------------------------------------------------------------------- '''
        if n_sub_clusters > 0:
            scl_centroids = []
            scl_labels = []
            scl_ncentroids =[]
            idx_centroids_sc= pd.DataFrame() #va almacenar los centroides de los subclusters
            #obtener los resultados del clusterizado
            for i in range(n_clusters):
                if subclusters_data=='wind':
                    dfclvvnoNA=dfclvv['C'+str(i+1)].dropna()
                    #buscando centroides con los subclusters
                    kmeans_sc = cluster.KMeans(n_clusters=n_sub_clusters, random_state=0).fit(dfclvvnoNA)
                    min_dist_sc = np.min(cdist(dfclvvnoNA.values, kmeans_sc.cluster_centers_, 'euclidean'), axis=1)
                    Y_sc = pd.DataFrame(min_dist_sc, index=dfclvvnoNA.index, columns=['PCTimeStamp'])
                    Z_sc = pd.DataFrame(kmeans_sc.labels_, index=dfclvvnoNA.index, columns=['subcluster_ID'])

                elif subclusters_data=='pow':
                    dfclvpnoNA=dfclvp['C'+str(i+1)].dropna()
                    #buscando centroides con los subclusters
                    kmeans_sc = cluster.KMeans(n_clusters=n_sub_clusters, random_state=0).fit(dfclvpnoNA)
                    min_dist_sc = np.min(cdist(dfclvpnoNA.values, kmeans_sc.cluster_centers_, 'euclidean'), axis=1)
                    Y_sc = pd.DataFrame(min_dist_sc, index=dfclvpnoNA.index, columns=['PCTimeStamp'])
                    Z_sc = pd.DataFrame(kmeans_sc.labels_, index=dfclvpnoNA.index, columns=['subcluster_ID'])
                else:
                    print('Todavía no programado, haciendo subclusters en potencia')
                    kmeans_sc = cluster.KMeans(n_clusters=n_sub_clusters, random_state=0).fit(dfclvp['C'+str(i+1)].dropna())

                scl_centroids.append(kmeans_sc.cluster_centers_)
                scl_labels.append(kmeans_sc.labels_)
                scl_ncentroids.append(len(scl_centroids[i]))


                PAP_sc = pd.concat([Y_sc,Z_sc], axis=1)
                #poniendo index a los centroides de los subclusters
                grouped_sc = PAP_sc.groupby(['subcluster_ID'])
                idx_cent_sc = grouped_sc.idxmin()
                idx_cent_sc.index +=1
                #agregando columnas de velocidadd de viento y potencia al dataframe de los centroides
                vxvy_sc=dfvxvy.loc[idx_cent_sc.PCTimeStamp].values
                vvpot_sc = dfVP.loc[idx_cent_sc.PCTimeStamp].values
                idx_centroids_sc=pd.concat([idx_centroids_sc, idx_cent_sc.assign(vx=vxvy_sc[:,0],vy=vxvy_sc[:,1],vwind=vvpot_sc[:,0], pw=vvpot_sc[:,1], cluster_ID='C'+str(i+1))] )
            #creando el multiindex fuera del ciclo for
            # idx_centroids_sc.set_index(['cluster_ID',idx_centroids_sc.index],inplace=True)

            idx_centroids_sc.reset_index(inplace=True)
            idx_centroids_sc.sort_values(['cluster_ID',pw_col_name], ascending=[1,1],inplace=True)
            lst_num_sc=list(range(1,n_sub_clusters+1))*n_clusters#crea una lista de numeros para el subcluster [1,2,3..1,2,3]
            sc_idx_list =list(map(lambda x:'SC'+str(x),lst_num_sc))#le pone las letras SC [SC1,SC2,SC3...SC1,SC2,SC3]
            idx_centroids_sc=idx_centroids_sc.assign(subcluster_ID_ord=sc_idx_list)
            idx_centroids_sc.set_index(['cluster_ID','subcluster_ID_ord'],inplace=True)

            ############  ORDENAR CENTROIDES:  ordenar el orden de aparicion segun la magnitud de la vv
            scl_magni = np.zeros([n_clusters,n_sub_clusters])
            scl_ord = []

            for i in range(n_clusters):

                for j in range(n_sub_clusters):

                    vx = dfclvv['C' + str(i+1)].vx.dropna().values[scl_labels[i] == j]
                    vy = dfclvv['C' + str(i+1)].vy.dropna().values[scl_labels[i] == j]
                    scl_magni[i][j] = np.mean(np.sqrt(vx**2 + vy**2))  #magnitud de la vv

                scl_ord.append( scl_magni[i].argsort())# ver https://github.com/numpy/numpy/issues/8757

###################  ORDENAR CENTROIDES ######################
            for i in range(len(scl_centroids)):
                scl_centroids[i] = scl_centroids[i][scl_ord[i]]

##################    CLUSTER VIENTO POTENCIA: #############
            colheadvp = []
            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    colheadvp.append(('C' + str(i + 1),'SC' + str(j + 1), wind_col_name))
                    colheadvp.append(('C' + str(i + 1),'SC' + str(j + 1), pw_col_name))
            dfsclvp = pd.DataFrame()

            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    dfsclvp = pd.concat([
                        dfsclvp, dfclvp['C'+str(i+1)].dropna()[wind_col_name][scl_labels[i]  == scl_ord[i][j]],
                        dfclvp['C'+str(i+1)].dropna()[pw_col_name][scl_labels[i] == scl_ord[i][j]]
                    ],
                        axis=1,
                        ignore_index=True)
            #crear multiindice
            dfsclvp.columns = pd.MultiIndex.from_tuples(colheadvp)

            #CLUSTER VX,VY:####################################################
            colheadvv = []
            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    colheadvv.append(('C' + str(i + 1),'SC' + str(j + 1), 'vx'))
                    colheadvv.append(('C' + str(i + 1),'SC' + str(j + 1), 'vy'))
            dfsclvv = pd.DataFrame()

            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    dfsclvv = pd.concat([
                        dfsclvv, dfclvv['C'+str(i+1)].dropna().vx[scl_labels[i]  == scl_ord[i][j]],
                        dfclvv['C'+str(i+1)].dropna().vy[scl_labels[i] == scl_ord[i][j]]
                    ],
                        axis=1,
                        ignore_index=True)
            #crear multiindice
            dfsclvv.columns = pd.MultiIndex.from_tuples(colheadvv)

            #CLUSTER POTENCIA:
            dfsclpw = pd.DataFrame()
            colheadpw =[]
            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    colheadpw.append(('C' + str(i + 1),'SC' + str(j + 1)))

            for i in range(n_clusters):
                for j in range(n_sub_clusters):
                    dfsclpw = pd.concat([dfsclpw,
                                         dfclvp['C'+str(i+1)][pw_col_name].dropna()[scl_labels[i] ==scl_ord[i][j]]],
                                        ignore_index=True,
                                        axis=1)
            dfsclpw.columns = pd.MultiIndex.from_tuples(colheadpw)

            #### buscar centroides para la grafica vv
            #obtener nombre de niveles
            n_sub_clusters = len(dfsclvv.columns.levels[1])
            #numero total de clusters incluidos los subclusters
            n_tot_clusters = n_sub_clusters*n_clusters
            # solo aplica cuando hay subclusters
            lev0 = dfsclvv.columns.get_level_values(0)
            lev1 = dfsclvv.columns.get_level_values(1)
            namcl = [(lev0[i],lev1[i]) for i in range(len(lev0))]
            colnames = namcl[::2]



        # llenado de datos
        self.kmeans_labels = kmeans.labels_
        self.comp_vel = dfclvv
        self.cl_ord = cl_ord
        self.idx_centroids = idx_centroids
        self.cl_centers = kmeans.cluster_centers_
        #self.dfclpw = dfclpw
        self.dfclvp= dfclvp

        self.scl_ord= scl_ord
        self.scl_centroids = scl_centroids
        self.idx_centroids_sc = idx_centroids_sc


###############################################################################################################
n_clusters = 5
n_subclu=3
clusters_wind_pow = KMData()
clusters_wind_pow.dataframe_to_cluster(
    df_comp_vel,
    n_clusters=n_clusters,
    n_sub_clusters=n_subclu,
    clusters_data='wind',
    subclusters_data='pw',
    dfVP=df_wind_pow_ideal,
    pw_col_name='pw',
    wind_col_name='vwind')

Todavía no programado, haciendo subclusters en potencia


UnboundLocalError: local variable 'Y_sc' referenced before assignment

# Dispersion de potencia (MAD)

In [29]:
n_clusters = 15
n_subclu=0
clusters_wind_pow = KMData()
clusters_wind_pow.dataframe_to_cluster(
    df_comp_vel,
    n_clusters=n_clusters,
    n_sub_clusters=n_subclu,
    clusters_data='wind',
    subclusters_data='pw',
    dfVP=df_wind_pow_ideal,
    pw_col_name='pw',
    wind_col_name='vwind')

In [43]:
#dispersión absoluta media
state_mad = []
for i in range(n_clusters):
    state_mad.append(clusters_wind_pow.dfclvp['C'+str(i+1)].vwind.mad())

In [44]:
state_mad

[0.4063224073826566,
 0.5366859045770763,
 0.7833830505159134,
 0.9449875541834434,
 0.5860336592368246,
 1.1027728617352337,
 1.461815902831721,
 0.6020825666200532,
 1.042448579055071,
 0.7412352980641068,
 0.89859637730721,
 0.5844422291186423,
 1.1081328827139818,
 0.7758926308527061,
 1.408367549046537]

In [45]:
clusters_wind_pow.dfclvp['C1'].vwind.std()

0.47765093754087284

In [46]:
clusters_wind_pow.idx_centroids

,cluster_ID,PCTimeStamp,vx,vy,vwind,pw
cluster_ID_ord,,,,,,
1,2,2005-01-30 00:40:00,0.047,0.090,0.100,0.000
2,12,2005-05-04 02:00:00,-0.757,1.780,1.900,0.000
3,4,2005-07-21 07:20:00,2.253,0.193,2.300,0.000
4,14,2005-03-29 20:30:00,0.360,-2.846,2.900,0.000
5,0,2005-10-25 06:00:00,-2.726,3.152,4.200,99200.000
6,6,2005-09-01 12:30:00,-4.103,-1.084,4.200,99200.000
7,13,2005-07-20 17:00:00,3.293,4.976,6.000,437000.000
8,11,2005-04-09 20:20:00,-3.427,5.341,6.300,534800.000
9,5,2005-11-28 16:40:00,3.283,-5.899,6.800,708600.000


In [47]:
df_mf_curve.plot()
plt.show()

<IPython.core.display.Javascript object>

## Potencias y slider gain

In [68]:
#Estado  (base) ( no necesariamiente el numero de estado coincide con el de idx_centroids por la agrupacion)
#el estado base es el 10 (fotografía del sistema)
g1 = 0.923*13 #granja 1 y 2
g2 = 0.833*12

#base de potencia
potencia_base = g1+g2 #MW
print('potencia g1 %f, potencia g2 %f, potencia g1+g2 (base) %f'%(g1,g2,potencia_base))

potencia g1 11.999000, potencia g2 9.996000, potencia g1+g2 (base) 21.995000


In [69]:
clusters_wind_pow.idx_centroids

,cluster_ID,PCTimeStamp,vx,vy,vwind,pw
cluster_ID_ord,,,,,,
1,2,2005-01-30 00:40:00,0.047,0.090,0.100,0.000
2,12,2005-05-04 02:00:00,-0.757,1.780,1.900,0.000
3,4,2005-07-21 07:20:00,2.253,0.193,2.300,0.000
4,14,2005-03-29 20:30:00,0.360,-2.846,2.900,0.000
5,0,2005-10-25 06:00:00,-2.726,3.152,4.200,99200.000
6,6,2005-09-01 12:30:00,-4.103,-1.084,4.200,99200.000
7,13,2005-07-20 17:00:00,3.293,4.976,6.000,437000.000
8,11,2005-04-09 20:20:00,-3.427,5.341,6.300,534800.000
9,5,2005-11-28 16:40:00,3.283,-5.899,6.800,708600.000


In [70]:
lista_estados =np.arange(1,n_clusters+1)
lista_vel = []
for i in lista_estados:
    lista_vel.append(clusters_wind_pow.idx_centroids.loc[i].vwind)

lista_pw=[]
for i,j in enumerate(lista_vel):
    lista_pw.append(df_mfcurve_interp.loc[j].pw)
    #lista_pw.append(clusters_wind_pow.idx_centroids[clusters_wind_pow.idx_centroids.vwind == j].pw)
    print('Estado: %i velocidad %.2f m/s = %E w'% (lista_estados[i],j,lista_pw[i]))

Estado: 1 velocidad 0.10 m/s = 0.000000E+00 w
Estado: 2 velocidad 1.90 m/s = 0.000000E+00 w
Estado: 3 velocidad 2.30 m/s = 0.000000E+00 w
Estado: 4 velocidad 2.90 m/s = 0.000000E+00 w
Estado: 5 velocidad 4.20 m/s = 9.920000E+04 w
Estado: 6 velocidad 4.20 m/s = 9.920000E+04 w
Estado: 7 velocidad 6.00 m/s = 4.370000E+05 w
Estado: 8 velocidad 6.30 m/s = 5.348000E+05 w
Estado: 9 velocidad 6.80 m/s = 7.086000E+05 w
Estado: 10 velocidad 7.30 m/s = 8.884000E+05 w
Estado: 11 velocidad 7.50 m/s = 9.600000E+05 w
Estado: 12 velocidad 8.90 m/s = 1.497200E+06 w
Estado: 13 velocidad 10.30 m/s = 1.888800E+06 w
Estado: 14 velocidad 11.10 m/s = 2.000000E+06 w
Estado: 15 velocidad 11.20 m/s = 2.000000E+06 w


In [85]:
lista_potg1_g2_sg = []#lista (tupla) de (estado,potencia granja1,potencia granja2,sg) slider gain
for idx,ne in enumerate(range(len(lista_estados))):
    lista_potg1_g2_sg.append((lista_estados[idx],lista_pw[idx]*13,lista_pw[idx]*12,
                             (potencia_base*1E6-((lista_pw[idx]*13)+(lista_pw[idx]*12)))/200E6))
    
for i in lista_potg1_g2_sg:
    print('Estado: %i, Potencia g1[Mw]: %f, Potencia g2[Mw]: %f, G1+G2; %f,  Slider Gain: %0.6f'%(i[0],i[1]/1E6,
                                                                                                  i[2]/1E6,
                                                                                                  i[1]/1E6 +
                                                                                                  i[2]/1E6, i[3]))

Estado: 1, Potencia g1[Mw]: 0.000000, Potencia g2[Mw]: 0.000000, G1+G2; 0.000000,  Slider Gain: 0.109975
Estado: 2, Potencia g1[Mw]: 0.000000, Potencia g2[Mw]: 0.000000, G1+G2; 0.000000,  Slider Gain: 0.109975
Estado: 3, Potencia g1[Mw]: 0.000000, Potencia g2[Mw]: 0.000000, G1+G2; 0.000000,  Slider Gain: 0.109975
Estado: 4, Potencia g1[Mw]: 0.000000, Potencia g2[Mw]: 0.000000, G1+G2; 0.000000,  Slider Gain: 0.109975
Estado: 5, Potencia g1[Mw]: 1.289600, Potencia g2[Mw]: 1.190400, G1+G2; 2.480000,  Slider Gain: 0.097575
Estado: 6, Potencia g1[Mw]: 1.289600, Potencia g2[Mw]: 1.190400, G1+G2; 2.480000,  Slider Gain: 0.097575
Estado: 7, Potencia g1[Mw]: 5.681000, Potencia g2[Mw]: 5.244000, G1+G2; 10.925000,  Slider Gain: 0.055350
Estado: 8, Potencia g1[Mw]: 6.952400, Potencia g2[Mw]: 6.417600, G1+G2; 13.370000,  Slider Gain: 0.043125
Estado: 9, Potencia g1[Mw]: 9.211800, Potencia g2[Mw]: 8.503200, G1+G2; 17.715000,  Slider Gain: 0.021400
Estado: 10, Potencia g1[Mw]: 11.549200, Potencia g2[

In [84]:
i[3]

-0.140025

In [82]:

#estado 14, slider gain
#potencia en estado 14,8.2ms 9.446000E+05 w * n aerogeneradores

pwE14g1=26E+06
pwE14g2=24E+06

#potencia base-potencia de la granja1 en el estado 14, 8.2
SGE14=(potencia_base*1E6-(pwE14g1+pwE14g2))/200E6 #SG control slider gain de simulink (desviación de potencia)
print('Potencia E14 granja 1: %f\nPotencia E14 granja 2: %f\nDesviacion de potencia SG: %0.6f'%(pwE14g1,pwE14g2,SGE14))

Potencia E14 granja 1: 26000000.000000
Potencia E14 granja 2: 24000000.000000
Desviacion de potencia SG: -0.140025


In [76]:
#estado 14, slider gain
#potencia en estado 14,8.2ms 9.446000E+05 w * n aerogeneradores
pwE14g1=9.446000E+05*25
pwE14g2=9.446000E+05*23

#potencia base-potencia de la granja1 en el estado 14, 8.2
SGE14=(potencia_base*1E6-pwE14g1)/200E6 #SG control slider gain de simulink (desviación de potencia)
print('Potencia E14 granja 1: %f\nPotencia E14 granja 2: %f\nDesviacion de potencia SG: %f'%(pwE14g1,pwE14g2,SGE14))

Potencia E14 granja 1: 23615000.000000
Potencia E14 granja 2: 21725800.000000
Desviacion de potencia SG: -0.008100


In [77]:
#estado 9, slider gain
#potencia en estado 9,5.3m/s 279915.905 * n aerogeneradores
pwE9g1=2.432000E+05*13
pwE9g2=2.432000E+05*12
SGE9=(potencia_base*1E6-(pwE9g1+pwE9g2))/200E6 #SG control slider gain de simulink (desviación de potencia)

#potencia base-potencia de la granja1 en el estado 14, 8.2
print('Potencia E9 granja 1: %f\nPotencia E9 granja 2: %f\nDesviacion de potencia SG: %f'%(pwE9g1,pwE9g2,SGE9))


Potencia E9 granja 1: 3161600.000000
Potencia E9 granja 2: 2918400.000000
Desviacion de potencia SG: 0.079575
